In [11]:
import pretty_midi
import numpy as np
import muspy
import os
%run parse.ipynb
# %run mlp_model.ipynb

In [ ]:
# Install soundfont for muspy
muspy.download_musescore_soundfont()

In [12]:
# Convert the expressive timings to a MIDI file then to an MP3 file
# @param:
    # tempoPath: path to the tempo file
    # sourceInputPath: path to the source input file
    # outputPath: path to the output file
    # name: name of the .mp3 file
def synthesizeOutput(tempoPath, sourceInputPath, outputPath, soundfontPath, name):
    tempo = parseTempoFile(tempoPath)
    MIDI = pretty_midi.PrettyMIDI(initial_tempo = tempo)

    # Create an Instrument instance for a cello instrument
    violin_program = pretty_midi.instrument_name_to_program('Violin')
    violin = pretty_midi.Instrument(program=violin_program)

    # Create notes
    source_input = parseInputFile(sourceInputPath)
    ground_truth = parseGroundtruthFile(outputPath)

    for notes, times in zip(source_input, ground_truth):
        note = pretty_midi.Note(
            velocity=64, pitch=int(notes[2]), start=times[0], end=times[1])
        violin.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    MIDI.instruments.append(violin)

    # Write out the MIDI data
    MIDI.write('violin.mid')

    # Convert WAV to MP3 using pydub
    os.system(f'fluidsynth -ni {soundfontPath} {"violin.mid"} -F {name + ".wav"} -r 44100')

    # Remove the MIDI file
    os.remove('violin.mid')

In [ ]:
def synthesizeSourceInput(mxlPath, name):
    m = muspy.read(mxlPath)
    m.write_audio(name + ".wav")